## Sources:
https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780 - GCN explanation <br />
https://docs.dgl.ai/ - dgl docs <br />
https://github.com/dmlc/dgl/blob/master/examples/pytorch/rgcn/link_predict.py - dgl link prediction example <br />
https://medium.com/towards-artificial-intelligence/a-gentle-introduction-to-graph-embeddings-c7b3d1db0fa8 - Graph Embeddings <br />
https://arxiv.org/pdf/1703.06103.pdf

In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/tochka')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from IPython.display import clear_output
!pip install dgl-cu101  
!pip install rdflib
clear_output()

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import dgl
from dgl.contrib.data import load_data
from dgl.nn.pytorch import RelGraphConv

import pandas as pd
import networkx as nx
import tqdm
import gc
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

import matplotlib.pyplot as plt
%matplotlib inline

Using backend: pytorch


In [0]:
class EmbeddingLayer(nn.Module):
    # h_dim - embedding dimension
    # num_models - vocabulary size
    def __init__(self, num_models, h_dim):
        super().__init__()
        self.embedding = torch.nn.Embedding(num_nodes, h_dim)
        
    def forward(self, g, h, r, norm):
        return self.embedding(h.squeeze())

# How does Graph Convolution Network works?

<b>More formally, a graph convolutional network (GCN) is a neural network that operates on graphs.</b><br/><br />

Given a graph $G = (V, E)$, a GCN takes as input<br />
- an input feature matrix $N × F⁰$ feature matrix ($X$), where $N$ is the number of nodes and $F⁰$ is the number of input features for each node, and
- an $N × N$ matrix representation of the graph structure such as the adjacency matrix $A$ of $G$.

<img src = "https://miro.medium.com/max/2000/1*pCeWhIrEFXoEgsB5eEB6sw.png"/>

A hidden layer in the GCN can thus be written as $Hⁱ = f(Hⁱ⁻¹, A))$ where $H⁰ = X$ and f is a propagation. Each layer $Hⁱ$ corresponds to an $N × Fⁱ$ feature matrix where each row is a feature representation of a node. At each layer, these features are aggregated to form the next layer’s features using the propagation rule $f$. In this way, features become increasingly more abstract at each consecutive layer. In this framework, variants of GCN differ only in the choice of propagation rule $f$.

<img src="https://tkipf.github.io/graph-convolutional-networks/images/gcn_web.png"/>

One of the simplest possible propagation rule is (<b>without self-loops and normalization, for understanding only</b>):

$f(Hⁱ, A) = σ(AHⁱWⁱ)$

where $Wⁱ$ is the weight matrix for layer i and σ is a non-linear activation function such as the ReLU function. The weight matrix has dimensions $Fⁱ$ × $Fⁱ⁺¹$



# R-GCN

The key difference between R-GCN and GCN is that in R-GCN, edges can represent different relations. In GCN, weight $W(l)$ in equation (1) is shared by all edges in layer $l$. In contrast, in R-GCN, different edge types use different weights and only edges of the same relation type $r$ are associated with the same projection weight $W(l)_r$.

\begin{split}h_i^{l+1} = \sigma\left(\sum_{j\in N_i}\frac{1}{c_i} W^{(l)} h_j^{(l)}\right)~~~~~~~~~~(1)\\\end{split}

\begin{split}h_i^{l+1} = \sigma\left(W_0^{(l)}h_i^{(l)}+\sum_{r\in R}\sum_{j\in N_i^r}\frac{1}{c_{i,r}}W_r^{(l)}h_j^{(l)}\right)~~~~~~~~~~(2)\\\end{split}

$h$ - hidden layer
$i$ - node index
$l$ - layer number

$c$ - normalization constant

where $N_i^r$ denotes the set of neighbor indices of node i under relation $r∈R$ and $c_i$,$r$ is a normalization constant. 

In [0]:
class RGCN(nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels, num_bases,
                num_hidden_layers=1, dropout=0, use_self_loop=False):
        super().__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels # num of relations
        self.num_bases = None if num_bases < 0 else num_bases
        self.num_hidden_layers = num_hidden_layers
        self.dropout = dropout
        self.use_self_loop = use_self_loop
        
        self.build_model()
    
    # creates rgcn layers
    def build_model(self):
        self.layers = nn.ModuleList()
        # i2h
        i2h = self.build_input_layer()
        if i2h is not None:
            self.layers.append(i2h)
        # h2h
        for idx in range(self.num_hidden_layers):
            h2h = self.build_hidden_layer(idx)
            self.layers.append(h2h)
        # h2o
        h2o = self.build_output_layer()
        if h2o is not None:
            self.layers.append(h2o)
    
    # input to hidden
    def build_input_layer(self):
        return EmbeddingLayer(self.num_nodes, self.h_dim)

    def build_hidden_layer(self, idx):
        act = F.relu if idx < self.num_hidden_layers - 1 else None
        
        """
        RelGraphConv parameters
        in_feat : int
            Input feature size.
        out_feat : int
            Output feature size.
        num_rels : int
            Number of relations.
        regularizer : str
            Which weight regularizer to use "basis" or "bdd"
        num_bases : int, optional
            Number of bases. If is none, use number of relations. Default: None.
        bias : bool, optional
            True if bias is added. Default: True
        activation : callable, optional
            Activation function. Default: None
        self_loop : bool, optional
            True to include self loop message. Default: False
        dropout : float, optional
        Dropout rate. Default: 0.0
        """
        return RelGraphConv(self.h_dim, self.h_dim, self.num_rels, "bdd",
                           self.num_bases, activation=act, self_loop=self.use_self_loop,
                           dropout=self.dropout)

    # hidden to output
    def build_output_layer(self):
        return None

    def forward(self, g, h, r, norm):
        for layer in self.layers:
            h = layer(g, h, r, norm)
        return h

# Link prediction

## RESCAL

RESCAL (Nickle et al., 2011) uses multiple matrics to represent the relations among entities. Assume that the total number of entity is $n$ while the total number of a relation is $m$, the total number of parameters is $n * n * m$. If there is no relation between an entity $i$ and entity $j$, the value is set to zero.

<img src="https://miro.medium.com/max/652/1*xXDqgwCAsykucFn5lRtKwg.png" />

## DistMult

Instead of use complex matrics, DistMult algo reduce the number of relations parameters by using a diagonal matrix only (i.e. restricted matrices). It requires fewer parameters for training. A number of parameters of RESCAL can be more than DistMult ten to a hundred times.

DistMult enjoys a low number of parameters (same as TransE) to achieve superior performance.

Our link prediction model can be regarded as an autoencoder  consisting  of  an  encoder:  an  R-GCN  producinglatent feature representations of entities, and a decoder: a tensor factorization model exploiting these representations to predict labeled edges (DistMult).

In [0]:
class LinkPredict(nn.Module):
    def __init__(self, in_dim, h_dim, num_rels, num_bases=-1,
                num_hidden_layers=1, dropout=0, reg_param=0):
        super().__init__()
        self.rgcn = RGCN(in_dim, h_dim, h_dim, num_rels * 2, num_bases,
                        num_hidden_layers, dropout)
        self.reg_param = reg_param
        self.w_relation = nn.Parameter(torch.Tensor(num_rels, h_dim))
        
        # nn.init.xavier_uniform_ - fills the input Tensor with values according to the method
        # described in "Understanding the difficulty of training deep
        # feedforward neural networks"
        #
        # Return the recommended gain value for the given nonlinearity function
        nn.init.xavier_uniform_(self.w_relation, gain=nn.init.calculate_gain("relu"))
    
    # Link  prediction  deals  with  prediction  of  new  facts
    # (i.e.triples(subject, relation, object)). Formally, the knowledgebase
    # is  represented  by  a  directed,  labeled  graphG=(V,E,R). Rather than
    # the full set of edges E, we are givenonly  an  incomplete  subsetˆE. 
    # The  task  is  to  assign  scores f(s,r,o) to  possible  edges (s,r,o) 
    # in  order  to  determinehow likely those edges are to belong to E
    def calc_score(self, embedding, triplets):
        # DistMult
        s = embedding[triplets[:, 0]]
        r = self.w_relation[triplets[:, 1]]
        o = embedding[triplets[:, 2]]
        score = torch.sum(s * r * o, dim=1)
        return score
    
    def forward(self, g, h, r, norm):
        return self.rgcn.forward(g, h, r, norm)
    
    def regularization_loss(self, embedding):
        return torch.mean(embedding.pow(2)) + torch.mean(self.w_relation.pow(2))
    
    def get_loss(self, g, embed, triplets, labels):
        # triplets is a list of data samples (positive and negative)
        # each row in the triplets is a 3-tuple of (source, relation, destination)
        score = self.calc_score(embed, triplets)
        predict_loss = F.binary_cross_entropy_with_logits(score, labels)
        reg_loss = self.regularization_loss(embed)

        print("SCORE: ", score)
        print("LABELS: ", labels)

        return predict_loss + self.reg_param * reg_loss


# Playing with data

In [0]:
edges = pd.read_csv("edges.csv")
ids = pd.read_csv("ids.csv")
vertices = pd.read_csv("vertices.csv")

In [0]:
 def read_regions():
    regions_f = open("regions.txt").read()
    regions_lines = regions_f.split("\n")
    
    res = {}    
    for line in regions_lines:
        els = line.split(" ")
        if len(els) < 3:
            continue

        code = int(els[0])
        x = float(els[-1])
        y = float(els[-2])
        name = " ".join(els[1:-3])
    
        res[code] = [x, y, name]

    return res

In [0]:
coords_by_region = read_regions()
coords_by_region[99] = [None, None, None, None]
coords_by_region[81] = [None, None, None, None]
coords_by_region[85] = [None, None, None, None]
coords_by_region[0] = [None, None, None, None]

In [0]:
vertices["region_x"] = vertices["region_code"].apply(lambda x: coords_by_region[x][0]) 
vertices["region_y"] = vertices["region_code"].apply(lambda x: coords_by_region[x][1]) 

In DGL all vertices are numerating from 0

In [0]:
decremented = False
if not decremented:
    edges["id_1"] = edges["id_1"] - 1
    edges["id_2"] = edges["id_2"] - 1
    vertices["id"] = vertices["id"] - 1
    decremented = True

In [12]:
edges.head()

,id_1,id_2,value,n_transactions
0,878326,1133996,478035.238733,277.747437
1,707355,1341540,442189.669684,80.997950
2,169981,494073,353097.929209,287.789650
3,551009,979932,537749.674840,426.743337
4,76063,597022,418990.198382,287.789650


In [0]:
def is_connected(id1, id2):
    return len(edges[(edges["id_1"] == id1) & (edges["id_2"] == id2)]) > 0 or \
        len(edges[(edges["id_1"] == id2) & (edges["id_2"] == id1)]) > 0

In [14]:
is_connected(597022, 76063)

True

In [0]:
featured_nodes_id = vertices.id.unique()
connected_nodes_id = pd.concat((edges.id_1, edges.id_2)).unique()

In [16]:
len(featured_nodes_id), len(connected_nodes_id)

(1534749, 1514713)

In [0]:
connected_featured_nodes_id = np.array(list(set(connected_nodes_id) & set(featured_nodes_id)))
train_nodes = vertices[vertices["id"].isin(connected_featured_nodes_id)].copy()

In [0]:
train_nodes.reset_index(drop=True, inplace=True)
train_nodes.reset_index(inplace=True)
train_nodes.rename(columns={"index": "new_id"}, inplace=True)

In [19]:
train_nodes.head()

,new_id,id,main_okved,region_code,company_type,region_x,region_y
0,0,0,46.75,77,Limited,37.085390,55.219390
1,1,1,41.20,78,Limited,29.700980,59.941800
2,2,2,25.11,50,Limited,37.327330,55.479205
3,3,3,45.31,89,Limited,74.341549,67.147163
4,4,4,56.10,50,Limited,37.327330,55.479205


In [0]:
old_ids = train_nodes["id"]
new_ids = train_nodes["new_id"]
get_new_index, get_old_index = {}, {}

for oi, ni in zip(old_ids, new_ids):
    get_new_index[oi] = ni
    get_old_index[ni] = oi

In [0]:
edges["new_id_1"] = edges["id_1"].apply(lambda x: get_new_index[x])
edges["new_id_2"] = edges["id_2"].apply(lambda x: get_new_index[x])

#Playing with graph

In [0]:
def build_trans_graph(ids):
    src = edges.loc[edges["new_id_1"].isin(ids), "new_id_1"]
    dst = edges.loc[edges["new_id_2"].isin(ids), "new_id_2"]
    
    # making bi-directional edges
    u = np.concatenate([src, dst])
    v = np.concatenate([dst, src])

    return dgl.DGLGraph((u, v))

In [0]:
graph = build_trans_graph(connected_featured_nodes_id)

In [0]:
train_nodes["company_type"] = train_nodes["company_type"].astype("category")
train_nodes["company_type_cat_code"] = train_nodes["company_type"].cat.codes

comp_enc = OneHotEncoder(handle_unknown='ignore')
train_nodes.reset_index(inplace=True, drop=True)
train_nodes = train_nodes.join(pd.DataFrame(comp_enc.fit_transform(train_nodes[["company_type_cat_code"]]).toarray(), columns=[f"comp_one_hot{i}" for i in range(6)]))

In [0]:
def normalize_feature(name):
    train_nodes[f"{name}_normalized"] = (train_nodes[name] - train_nodes[name].min()) /\
        (train_nodes[name].max() - train_nodes[name].min())

normalize_feature("main_okved")
normalize_feature("region_x")
normalize_feature("region_y")

In [26]:
train_nodes.head()

,new_id,id,main_okved,region_code,company_type,region_x,region_y,company_type_cat_code,comp_one_hot0,comp_one_hot1,comp_one_hot2,comp_one_hot3,comp_one_hot4,comp_one_hot5,main_okved_normalized,region_x_normalized,region_y_normalized
0,0,0,46.75,77,Limited,37.085390,55.219390,3,0.0,0.0,0.0,1.0,0.0,0.0,0.466837,0.107431,0.501156
1,1,1,41.20,78,Limited,29.700980,59.941800,3,0.0,0.0,0.0,1.0,0.0,0.0,0.410204,0.057651,0.685477
2,2,2,25.11,50,Limited,37.327330,55.479205,3,0.0,0.0,0.0,1.0,0.0,0.0,0.246020,0.109062,0.511297
3,3,3,45.31,89,Limited,74.341549,67.147163,3,0.0,0.0,0.0,1.0,0.0,0.0,0.452143,0.358583,0.966711
4,4,4,56.10,50,Limited,37.327330,55.479205,3,0.0,0.0,0.0,1.0,0.0,0.0,0.562245,0.109062,0.511297


In [0]:
graph.ndata["company_type"] = train_nodes.loc[:, [f"comp_one_hot{i}" for i in range(6)]].to_numpy()
graph.ndata["coords"] = train_nodes.loc[:, ["region_x", "region_y"]].to_numpy()

# RGCNData implementation

In [0]:
def _read_triplets(filename):
    with open(filename, 'r+') as f:
        for line in f:
            processed_line = line.strip().split('\t')
            yield processed_line

def _read_triplets_as_list(filename, entity_dict, relation_dict):
    l = []
    for triplet in _read_triplets(filename):
        s = entity_dict[triplet[0]]
        r = relation_dict[triplet[1]]
        o = entity_dict[triplet[2]]
        l.append([s, r, o])
    return l

def _read_dictionary(filename):
    d = {}
    with open(filename, 'r+') as f:
        for line in f:
            line = line.strip().split('\t')
            d[line[1]] = int(line[0])
    return d


class RGCNLinkDataset(object):
    """RGCN link prediction dataset
    The dataset contains a graph depicting the connectivity of a knowledge
    base. Currently, the knowledge bases from the
    `RGCN paper <https://arxiv.org/pdf/1703.06103.pdf>`_ supported are
    FB15k-237, FB15k, wn18
    The original knowledge base is stored as an RDF file, and this class will
    download and parse the RDF file, and performs preprocessing.
    An object of this class has 5 member attributes needed for link
    prediction:
    num_nodes: int
        number of entities of knowledge base
    num_rels: int
        number of relations (including reverse relation) of knowledge base
    train: numpy.array
        all relation triplets (src, rel, dst) for training
    valid: numpy.array
        all relation triplets (src, rel, dst) for validation
    test: numpy.array
        all relation triplets (src, rel, dst) for testing
    Usually, user don't need to directly use this class. Instead, DGL provides
    wrapper function to load data (see example below).
    Examples
    --------
    """
    def __init__(self, name):
        self.name = name
        self.dir = get_download_dir()
        tgz_path = os.path.join(self.dir, '{}.tar.gz'.format(self.name))
        download(_downlaod_prefix + '{}.tgz'.format(self.name), tgz_path)
        self.dir = os.path.join(self.dir, self.name)
        extract_archive(tgz_path, self.dir)

    def load(self):
        entity_path = os.path.join(self.dir, 'entities.dict')
        relation_path = os.path.join(self.dir, 'relations.dict')
        train_path = os.path.join(self.dir, 'train.txt')
        valid_path = os.path.join(self.dir, 'valid.txt')
        test_path = os.path.join(self.dir, 'test.txt')
        entity_dict = _read_dictionary(entity_path)
        relation_dict = _read_dictionary(relation_path)
        self.train = np.asarray(_read_triplets_as_list(train_path, entity_dict, relation_dict))
        self.valid = np.asarray(_read_triplets_as_list(valid_path, entity_dict, relation_dict))
        self.test = np.asarray(_read_triplets_as_list(test_path, entity_dict, relation_dict))
        print("Train: ", self.train)
        print("Valid: ", self.valid)
        print("Test: ", self.test)
        self.num_nodes = len(entity_dict)
        print("# entities: {}".format(self.num_nodes))
        self.num_rels = len(relation_dict)
        print("# relations: {}".format(self.num_rels))
        print("# edges: {}".format(len(self.train)))


class CustomRGCNLinkDataset(object):
    def __init__(self, entities_dict_path, relations_dict_path, train_path,
                 valid_path, test_path):
        self.entities_dict_path = entities_dict_path
        self.relations_dict_path = relations_dict_path
        self.train_path = train_path
        self.valid_path = valid_path
        self.test_path = test_path

    
    def load(self):
        entity_dict = _read_dictionary(self.entities_dict_path)
        relation_dict = _read_dictionary(self.relations_dict_path)
        self.train = np.asarray(_read_triplets_as_list(self.train_path, entity_dict, relation_dict))
        self.valid = np.asarray(_read_triplets_as_list(self.valid_path, entity_dict, relation_dict))
        self.test = np.asarray(_read_triplets_as_list(self.test_path, entity_dict, relation_dict))

        print("Train: ", self.train)
        print("Valid: ", self.valid)
        print("Test: ", self.test)
        self.num_nodes = len(entity_dict)
        print("# entities: {}".format(self.num_nodes))
        self.num_rels = len(relation_dict)
        print("# relations: {}".format(self.num_rels))
        print("# edges: {}".format(len(self.train)))


def load_link(dataset):
    data = RGCNLinkDataset(dataset)
    data.load()
    return data

def load_custom_link():
    data = CustomRGCNLinkDataset(entities_dict_path="entities.dict", relations_dict_path="relations.dict",
                                 train_path="train_data.txt", test_path="test_data.txt", valid_path="valid_data.txt")
    data.load()
    return data

# Loading example dataset

In [0]:
def comp_deg_norm(g):
    g = g.local_var()
    in_deg = g.in_degrees(range(g.number_of_nodes())).float().numpy()
    norm = 1.0 / in_deg
    norm[np.isinf(norm)] = 0
    return norm

def get_adj_and_degrees(num_nodes, triplets):
    """ Get adjacency list and degrees of the graph
    """
    adj_list = [[] for _ in range(num_nodes)]
    for i,triplet in enumerate(triplets):
        adj_list[triplet[0]].append([i, triplet[2]])
        adj_list[triplet[2]].append([i, triplet[0]])

    degrees = np.array([len(a) for a in adj_list])
    adj_list = [np.array(a) for a in adj_list]
    return adj_list, degrees


def build_graph_from_triplets(num_nodes, num_rels, triplets):
    """ Create a DGL graph. The graph is bidirectional because RGCN authors
        use reversed relations.
        This function also generates edge type and normalization factor
        (reciprocal of node incoming degree)
    """
    g = dgl.DGLGraph()
    g.add_nodes(num_nodes)
    src, rel, dst = triplets
    src, dst = np.concatenate((src, dst)), np.concatenate((dst, src))
    rel = np.concatenate((rel, rel + num_rels))
    edges = sorted(zip(dst, src, rel))
    dst, src, rel = np.array(edges).transpose()
    g.add_edges(src, dst)
    norm = comp_deg_norm(g)
    print("# nodes: {}, # edges: {}".format(num_nodes, len(src)))
    return g, rel.astype('int64'), norm.astype('int64')


def build_test_graph(num_nodes, num_rels, edges):
    src, rel, dst = edges.transpose()
    print("Test graph:")
    return build_graph_from_triplets(num_nodes, num_rels, (src, rel, dst))

def negative_sampling(pos_samples, num_entity, negative_rate):
    size_of_batch = len(pos_samples)
    num_to_generate = size_of_batch * negative_rate
    neg_samples = np.tile(pos_samples, (negative_rate, 1))
    labels = np.zeros(size_of_batch * (negative_rate + 1), dtype=np.float32)
    labels[: size_of_batch] = 1
    values = np.random.randint(num_entity, size=num_to_generate)
    choices = np.random.uniform(size=num_to_generate)
    subj = choices > 0.5
    obj = choices <= 0.5
    neg_samples[subj, 0] = values[subj]
    neg_samples[obj, 2] = values[obj]

    return np.concatenate((pos_samples, neg_samples)), labels


def sample_edge_neighborhood(adj_list, degrees, n_triplets, sample_size):
    """Sample edges by neighborhool expansion.
    This guarantees that the sampled edges form a connected graph, which
    may help deeper GNNs that require information from more than one hop.
    """
    edges = np.zeros((sample_size), dtype=np.int32)

    #initialize
    sample_counts = np.array([d for d in degrees])
    picked = np.array([False for _ in range(n_triplets)])
    seen = np.array([False for _ in degrees])

    for i in range(0, sample_size):
        weights = sample_counts * seen

        if np.sum(weights) == 0:
            weights = np.ones_like(weights)
            weights[np.where(sample_counts == 0)] = 0

        probabilities = (weights) / np.sum(weights)
        chosen_vertex = np.random.choice(np.arange(degrees.shape[0]),
                                         p=probabilities)
        chosen_adj_list = adj_list[chosen_vertex]
        seen[chosen_vertex] = True

        chosen_edge = np.random.choice(np.arange(chosen_adj_list.shape[0]))
        chosen_edge = chosen_adj_list[chosen_edge]
        edge_number = chosen_edge[0]

        while picked[edge_number]:
            chosen_edge = np.random.choice(np.arange(chosen_adj_list.shape[0]))
            chosen_edge = chosen_adj_list[chosen_edge]
            edge_number = chosen_edge[0]

        edges[i] = edge_number
        other_vertex = chosen_edge[1]
        picked[edge_number] = True
        sample_counts[chosen_vertex] -= 1
        sample_counts[other_vertex] -= 1
        seen[other_vertex] = True
    return edges


def sample_edge_uniform(adj_list, degrees, n_triplets, sample_size):
    """Sample edges uniformly from all the edges."""
    all_edges = np.arange(n_triplets)
    return np.random.choice(all_edges, sample_size, replace=False)


def generate_sampled_graph_and_labels(triplets, sample_size, split_size,
                                      num_rels, adj_list, degrees,
                                      negative_rate, sampler="uniform"):
    """Get training graph and signals
    First perform edge neighborhood sampling on graph, then perform negative
    sampling to generate negative samples
    """
    # perform edge neighbor sampling
    if sampler == "uniform":
        edges = sample_edge_uniform(adj_list, degrees, len(triplets), sample_size)
    elif sampler == "neighbor":
        edges = sample_edge_neighborhood(adj_list, degrees, len(triplets), sample_size)
    else:
        raise ValueError("Sampler type must be either 'uniform' or 'neighbor'.")

    # relabel nodes to have consecutive node ids
    edges = triplets[edges]
    src, rel, dst = edges.transpose()
    uniq_v, edges = np.unique((src, dst), return_inverse=True)
    src, dst = np.reshape(edges, (2, -1))
    relabeled_edges = np.stack((src, rel, dst)).transpose()

    # negative sampling
    samples, labels = negative_sampling(relabeled_edges, len(uniq_v),
                                        negative_rate)

    # further split graph, only half of the edges will be used as graph
    # structure, while the rest half is used as unseen positive samples
    split_size = int(sample_size * split_size)
    graph_split_ids = np.random.choice(np.arange(sample_size),
                                       size=split_size, replace=False) # here was replace=False
    src = src[graph_split_ids]
    dst = dst[graph_split_ids]
    rel = rel[graph_split_ids]

    # build DGL graph
    print("# sampled nodes: {}".format(len(uniq_v)))
    print("# sampled edges: {}".format(len(src) * 2))
    g, rel, norm = build_graph_from_triplets(len(uniq_v), num_rels,
                                             (src, rel, dst))
    return g, uniq_v, rel, norm, samples, labels

def node_norm_to_edge_norm(g, node_norm):
    g = g.local_var()
    # convert to edge norm
    g.ndata['norm'] = node_norm
    g.apply_edges(lambda edges : {'norm' : edges.dst['norm']})
    return g.edata['norm']

In [0]:
# example_data = load_data(dataset='wn18')
# example_num_nodes = example_data.num_nodes
# example_train_data = example_data.train
# example_valid_data = example_data.valid
# example_test_data = example_data.test
# example_num_rels = example_data.num_rels

# example_valid_data = torch.LongTensor(example_valid_data)
# example_test_data = torch.LongTensor(example_test_data)

In [0]:
# example_adj_list, example_degrees = get_adj_and_degrees(example_num_nodes, example_train_data)

### Generating sampled graph
``` 
       g, node_id, edge_type, node_norm, data, labels = \
            utils.generate_sampled_graph_and_labels(
                train_data, args.graph_batch_size, args.graph_split_size,
                num_rels, adj_list, degrees, args.negative_sample,
                args.edge_sampler)
```




In [0]:
# example_g, example_node_id, example_edge_type, example_node_norm, example_data, example_labels = \
#             generate_sampled_graph_and_labels(
#                 example_train_data, 3000, 0.5,
#                 example_num_rels, example_adj_list, example_degrees, 10,
#                 "neighbor")

In [0]:
# np.unique(example_edge_type)

In [0]:
# example_g

In [0]:
# np.unique(example_labels)

In [0]:
# del example_g, example_node_id, example_edge_type, example_node_norm, example_data, example_labels

In [0]:
# gc.collect()

# Formatting our data

In [0]:
rewrite_data = False

In [0]:
if rewrite_data:
    open("relations.dict", "w").write("0\tconnected\n")

    np.savetxt("entities.dict", vertices.loc[:, "id"].reset_index().values, delimiter="\t", fmt="%s")
    formatted_data = pd.concat([edges.id_1.astype(np.int32), pd.Series(["connected"] * len(edges)),
                                edges.id_2.astype(np.int32)], axis=1, sort=False)

    train_p = int(len(formatted_data) * 0.6)
    test_p = train_p + int(len(formatted_data) * 0.2)

    train_formatted_data = formatted_data[:train_p]
    test_formatted_data = formatted_data[train_p:test_p]
    valid_formatted_data = formatted_data[test_p:]

    np.savetxt("train_data.txt", train_formatted_data.values, delimiter="\t", fmt="%s")
    np.savetxt("test_data.txt", test_formatted_data.values, delimiter="\t", fmt="%s")
    np.savetxt("valid_data.txt", valid_formatted_data.values, delimiter="\t", fmt="%s")

# Training

In [40]:
data = load_custom_link()
num_nodes = data.num_nodes
train_data = data.train
valid_data = data.valid
test_data = data.test
num_rels = data.num_rels

valid_data = torch.LongTensor(valid_data)
test_data = torch.LongTensor(test_data)

Train:  [[ 878326       0 1133996]
 [ 707355       0 1341540]
 [ 169981       0  494073]
 ...
 [ 486745       0 1531874]
 [ 426141       0 1244684]
 [ 286348       0  852482]]
Valid:  [[ 445402       0 1424122]
 [ 106459       0 1520094]
 [ 672069       0 1525340]
 ...
 [  45802       0  829030]
 [1266549       0 1448477]
 [  44950       0  995848]]
Test:  [[ 571990       0  861753]
 [ 550028       0 1200093]
 [ 125126       0 1129041]
 ...
 [ 676943       0  959199]
 [ 631475       0 1322747]
 [ 363969       0 1311511]]
# entities: 1534749
# relations: 1
# edges: 2811386


In [41]:
epochs = 2000

n_bases = 100
n_layers = 2
dropout = 0.2
reg_param = 0.01
n_hidden = 200 # 500

lr = 1e-2

graph_batch_size = 10000 # 30000
graph_split_size = 0.5
negative_sample = 10
edge_sampler = "uniform"

grad_norm = 1.0
evaluate_every = 100000000000

DEBUG = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [42]:
model = LinkPredict(num_nodes, n_hidden, num_rels, num_bases=n_bases,
                   num_hidden_layers=n_layers, dropout=dropout, reg_param=reg_param)
model.to(device)

LinkPredict(
  (rgcn): RGCN(
    (layers): ModuleList(
      (0): EmbeddingLayer(
        (embedding): Embedding(1534749, 200)
      )
      (1): RelGraphConv(
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (2): RelGraphConv(
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
  )
)

In [43]:
# build test graph
test_graph, test_rel, test_norm = build_test_graph(num_nodes, num_rels, train_data)
test_deg = test_graph.in_degrees(range(test_graph.number_of_nodes())).float().view(-1,1)
test_node_id = torch.arange(0, num_nodes, dtype=torch.long).view(-1, 1)
test_rel = torch.from_numpy(test_rel)
test_norm = node_norm_to_edge_norm(test_graph, torch.from_numpy(test_norm).view(-1, 1))

Test graph:


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


# nodes: 1534749, # edges: 5622772


In [0]:
adj_list, degrees = get_adj_and_degrees(num_nodes, train_data)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:
def log(*args):
    if DEBUG:
        print("DEBUG: ", *args)

In [46]:
for epoch in range(epochs):
    print(f"EPOCH {epoch}/{epochs}")
    model.train()
    
    # perform edge neighborhood sampling to generate training graph and data
    print("Sampling...")
    g, node_id, edge_type, node_norm, data, labels = \
            generate_sampled_graph_and_labels(
                train_data, graph_batch_size, graph_split_size,
                num_rels, adj_list, degrees, negative_sample,
                edge_sampler)
    
    # set node/edge feature
    print("Extracting features...")
    node_id = torch.from_numpy(node_id).view(-1, 1).long().to(device)
    edge_type = torch.from_numpy(edge_type).to(device)
    edge_norm = node_norm_to_edge_norm(g, torch.from_numpy(node_norm).view(-1, 1)).to(device)
    data, labels = torch.from_numpy(data).to(device), torch.from_numpy(labels).to(device)
    deg = g.in_degrees(range(g.number_of_nodes())).float().view(-1, 1).to(device)
    
    embed = model(g, node_id, edge_type, edge_norm)

    log("G: ", g)
    log("Node id: ", node_id)
    log("Edge type: ", edge_type)
    log("Edge norm: ", edge_norm)

    loss = model.get_loss(g, embed, data, labels)

    log("Embed: ", embed)
    log("Data: ", data)
    log("Labels: ", labels)

    loss.backward()
    print(f"Loss: {loss}")
    
    torch.nn.utils.clip_grad_norm_(model.parameters(), grad_norm) # clip gradients
    optimizer.step()
    
    optimizer.zero_grad()

    # validation
    if (epoch + 1) % evaluate_every == 0:
        # perform validation on CPU because full graph is too large
        if torch.cuda.is_available():
            model.cpu()
        model.eval()
        print("start eval")
        embed = model(test_graph, test_node_id, test_rel, test_norm)
        mrr = utils.calc_mrr(embed, model.w_relation, torch.LongTensor(train_data),
                             valid_data, test_data, hits=[1, 3, 10], eval_bz=args.eval_batch_size,
                             eval_p=args.eval_protocol)
        # save best model
        if mrr < best_mrr:
            if epoch >= args.n_epochs:
                break
        else:
            best_mrr = mrr
            torch.save({'state_dict': model.state_dict(), 'epoch': epoch},
                       model_state_file)
        if torch.cuda.is_available():
            model.to(device)

EPOCH 0/2000
Sampling...
# sampled nodes: 18033
# sampled edges: 10000
# nodes: 18033, # edges: 10000
Extracting features...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.


Streaming output truncated to the last 5000 lines.
        [ 8525,     0, 14781],
        [ 2828,     0,  7154],
        ...,
        [ 5389,     0, 17283],
        [15319,     0,  8756],
        [10086,     0,  9013]])
DEBUG:  Labels:  tensor([1., 1., 1.,  ..., 0., 0., 0.])
Loss: 0.24808475375175476
EPOCH 1218/2000
Sampling...
# sampled nodes: 18093
# sampled edges: 10000
# nodes: 18093, # edges: 10000
Extracting features...
DEBUG:  G:  DGLGraph(num_nodes=18093, num_edges=10000,
         ndata_schemes={}
         edata_schemes={})
DEBUG:  Node id:  tensor([[    194],
        [    246],
        [    296],
        ...,
        [1534023],
        [1534428],
        [1534576]])
DEBUG:  Edge type:  tensor([1, 1, 1,  ..., 0, 0, 0])
DEBUG:  Edge norm:  tensor([[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]])
SCORE:  tensor([-1.9054, -2.9014, -1.1331,  ..., -1.9029, -3.9907, -4.7116],
       grad_fn=<SumBackward1>)
LABELS:  tensor([1., 1., 1.,  ..., 0., 0., 

KeyboardInterrupt: ignored

In [0]:
torch.save(model.state_dict(), "LinkPrediction.pt")

In [48]:
model

LinkPredict(
  (rgcn): RGCN(
    (layers): ModuleList(
      (0): EmbeddingLayer(
        (embedding): Embedding(1534749, 200)
      )
      (1): RelGraphConv(
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (2): RelGraphConv(
        (dropout): Dropout(p=0.2, inplace=False)
      )
    )
  )
)

In [51]:
node_id, edge_type = np.array([1]), np.array([0])
edge_type = torch.from_numpy(edge_type).to(device)
edge_norm = node_norm_to_edge_norm(g, torch.from_numpy(node_norm).view(-1, 1)).to(device)
data, labels = torch.from_numpy(data).to(device), torch.from_numpy(labels).to(device)
deg = g.in_degrees(range(g.number_of_nodes())).float().view(-1, 1).to(device)

embed = model(g, node_id, edge_type, edge_norm)

TypeError: ignored